In [1]:
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from scipy import stats

In [2]:
def get_even_num(num=63):
    runs = []
    evens = []
    j = 0
    for i in range(num):
        if (i != 0) & (i != 2):
            if i % 2 == 0:
                evens.append(str(i))
                j += 1
                if j == 5: 
                    runs.append(evens)
                    evens = []
                    j = 0
    return runs

In [3]:
def get_minus_num(base_runs):
    li = []
    for i in range(len(base_runs)):
        li.append([[base_runs[i][1],base_runs[i][0]], [base_runs[i][3],base_runs[i][0]], [base_runs[i][1],base_runs[i][3]]])
    return li

In [5]:
data_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/"
# data_path = "/Users/mriworkshop/Documents/TDCS/code/tdcs_thesis/data/"
save_path = data_path+"raw/"
sub_path_1_6 = data_path+"external/R21_SUB01_SES06"
sub_path_3_3 = data_path+"external/R21_SUB03_SES03"
sub_path_4_1 = data_path+"external/R21_SUB04_SES01"
#cwd_path = sub_path+"/NIFTI/bmaps_numpy/"
# tes = [2.46, 4.61, 7.09, 10.5, 15.0, 20.0, 24.5, 29.0, 33.5, 38.0, 42.5, 47.0]

base_runs = get_even_num()
bp = get_minus_num(base_runs)

In [6]:
base_runs

[['4', '6', '8', '10', '12'],
 ['14', '16', '18', '20', '22'],
 ['24', '26', '28', '30', '32'],
 ['34', '36', '38', '40', '42'],
 ['44', '46', '48', '50', '52'],
 ['54', '56', '58', '60', '62']]

In [7]:
bp

[[['6', '4'], ['10', '4'], ['6', '10']],
 [['16', '14'], ['20', '14'], ['16', '20']],
 [['26', '24'], ['30', '24'], ['26', '30']],
 [['36', '34'], ['40', '34'], ['36', '40']],
 [['46', '44'], ['50', '44'], ['46', '50']],
 [['56', '54'], ['60', '54'], ['56', '60']]]

# Extract Data

In [9]:
# Without theory data for prediction
def extract_mean_separate_neg(sub_path, experiment):
    print(sub_path)
    cwd_path = sub_path+"/bmaps/"
    result = pd.DataFrame()
    for m in range(len(base_runs)): # mini experiment
        # Bvals
#         for l in range(2):
#             tdir = cwd_path+bp[m][l][0]+"minus"+bp[m][l][1]+"/"
#             theory_img = nib.load(tdir+"Bfieldz_wJ_nT.nii")
#             theory_data = theory_img.get_data()
#             sp = theory_data.shape
#             b_data = []
            
#             neg = 0
#             if l==1:
#                 neg = 1

#             for i in range(sp[0]):
#                 for j in range(sp[1]):
#                     for k in range(sp[2]):
#                         b_data.append([experiment, m+1, i, j, k, neg, theory_data[i][j][k]])
            
#             if l==0:
#                 b_df = pd.DataFrame(b_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'neg', 'theory'])
#             else:
#                 b_df2 = pd.DataFrame(b_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'neg', 'theory'])

        df = pd.DataFrame()
        df2 = pd.DataFrame()
        new_df = pd.DataFrame()
        new_df2 = pd.DataFrame()
        for n in range(len(base_runs[m])): # 5 runs
            bdir = cwd_path+"base_run_"+base_runs[m][n]+"/"
            print(base_runs[m][n])

            new_data = []    
            new_data2 = [] 
            fmap_img = nib.load(bdir+"4d_fmap_Mean.nii.gz")
            fmap_data = fmap_img.get_data()
            sp = fmap_data.shape  
       
            for i in range(sp[0]):
                for j in range(sp[1]):
                    for k in range(sp[2]):
                        new_data.append([experiment, m+1, i, j, k, 0, fmap_data[i][j][k]])
                        new_data2.append([experiment, m+1, i, j, k, 1, fmap_data[i][j][k]])

            if n<=2:          
                col = "mean"+str(n)
                new_df = pd.DataFrame(new_data, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'neg', col])
            if n>=2:
                col = "mean"+str(n-2)
                new_df2 = pd.DataFrame(new_data2, columns=['exp', 'mini_exp', 'i', 'j', 'k', 'neg', col])
                
            if n == 0:
                df = new_df 
            elif n<=2:
                df = df.merge(new_df, on=['exp', 'mini_exp', 'i', 'j', 'k', 'neg'], sort=False)

            if n == 2:
                df2 = new_df2
            elif n>2:
                df2 = df2.merge(new_df2, on=['exp', 'mini_exp', 'i', 'j', 'k', 'neg'], sort=False)

#         df = df.merge(b_df, on=['exp','mini_exp', 'i', 'j', 'k', 'neg'], sort=False)
#         df2 = df2.merge(b_df2, on=['exp','mini_exp', 'i', 'j', 'k', 'neg'], sort=False)
        
        result = result.append(df, ignore_index=True)
        result = result.append(df2, ignore_index=True)
    return result

In [10]:
result_1_6 = extract_mean_separate_neg(sub_path_1_6, '0106')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_SUB01_SES06
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62


In [13]:
br_ = base_runs
bp_ = bp

base_runs = br_[:2]
bp = bp_[:2]

In [14]:
base_runs

[['4', '6', '8', '10', '12'], ['14', '16', '18', '20', '22']]

In [15]:
bp

[[['6', '4'], ['10', '4'], ['6', '10']],
 [['16', '14'], ['20', '14'], ['16', '20']]]

In [16]:
result_3_3 = extract_mean_separate_neg(sub_path_3_3, '0303')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_SUB03_SES03
4
6
8
10
12
14
16
18
20
22


In [17]:
base_runs = br_[:3]
bp = bp_[:3]

In [18]:
base_runs

[['4', '6', '8', '10', '12'],
 ['14', '16', '18', '20', '22'],
 ['24', '26', '28', '30', '32']]

In [19]:
result_4_1 = extract_mean_separate_neg(sub_path_4_1, '0401')

/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/data/external/R21_SUB04_SES01
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32


In [20]:
print(result_1_6.shape)
print(result_3_3.shape)
print(result_4_1.shape)

(2162688, 9)
(720896, 9)
(1081344, 9)


In [21]:
result = result_1_6.append(result_3_3, ignore_index=True)
result = result.append(result_4_1, ignore_index=True)
result

,exp,mini_exp,i,j,k,neg,mean0,mean1,mean2
0,0106,1,0,0,0,0,0.0,0.0,0.0
1,0106,1,0,0,1,0,0.0,0.0,0.0
2,0106,1,0,0,2,0,0.0,0.0,0.0
3,0106,1,0,0,3,0,0.0,0.0,0.0
4,0106,1,0,0,4,0,0.0,0.0,0.0
5,0106,1,0,0,5,0,0.0,0.0,0.0
6,0106,1,0,0,6,0,0.0,0.0,0.0
7,0106,1,0,0,7,0,0.0,0.0,0.0
8,0106,1,0,0,8,0,0.0,0.0,0.0
9,0106,1,0,0,9,0,0.0,0.0,0.0


In [22]:
np.savetxt(save_path+"fmap_mean_humans.txt", result.values, fmt="%s")